# Lab: Ingesting and Analyzing Avalanche Shipping Logs

**Objective:**  
You’ll use Snowflake and Python to ingest a CSV file containing Avalanche shipping logs. Then you will clean the data, and extract useful insights that can inform the next stage of your GenAI prototype.

**Prerequisite:**  
Make sure that you've already gone through the following prerequisite notebooks of Module 2 before proceeding with this lab's notebook:
- [M2L1V3.ipynb](https://github.com/dataprofessor/fast-prototyping-of-genai-apps-with-streamlit/blob/main/M2/Lesson_01/M2L1V3.ipynb)
- [M2L1V4.ipynb](https://github.com/dataprofessor/fast-prototyping-of-genai-apps-with-streamlit/blob/main/M2/Lesson_01/M2L1V4.ipynb)
- [M2L1V5.ipynb](https://github.com/dataprofessor/fast-prototyping-of-genai-apps-with-streamlit/blob/main/M2/Lesson_01/M2L1V5.ipynb)
- [M2L1V6.ipynb](https://github.com/dataprofessor/fast-prototyping-of-genai-apps-with-streamlit/blob/main/M2/Lesson_01/M2L1V6.ipynb)

**Story Context:**  
You just got your hands on internal shipping logs from Avalanche's distribution warehouse. These logs contain important operational data — delivery errors, shipping times, product IDs, and destinations.

Your job is to:
1. Upload the CSV file into Snowflake.
2. Clean and explore the data.
3. Save the data to a table in the database.

## ✅ Step 1: Upload the file to Snowflake Notebook

In [ ]:
from snowflake.snowpark.context import get_active_session

session = get_active_session()


# Read the shipping log CSV file from the Snowflake stage with headers
shipping_df = session.read.options({
    "inferSchema": True,
    "header": True
}).csv("@AVALANCHE_STAGE/shipping_logs.csv")
# Preview the data
shipping_df.show()

## ✅ Step 2: Clean and explore the data

Start by renaming the columns. Looking at the dataframe preview above, you can see that the column names have quotes around them. That's because snowpark loads column names exactly as they appear in the file—including quotes, capital letters, and spaces. The easy way to fix that is to just update them with an alias that renames the columns.


In [ ]:
from snowflake.snowpark.functions import col

# Rename columns explicitly (remove any extra characters or spaces)
shipping_df = shipping_df.select(
    col('"Order ID"').alias("Order_ID"),
    col('"Shipping Date"').alias("Shipping_Date"),
    col('"Carrier"').alias("Carrier"),
    col('"Tracking Number"').alias("Tracking_Number"),
    col('"Latitude"').alias("Latitude"),
    col('"Longitude"').alias("Longitude"),
    col('"Status"').alias("Status"),
    col('"Delivery Days"').alias("Delivery_Days"),
    col('"Late"').alias("Late"),
    col('"Region"').alias("Region")
)

Now you can count how many shipments each carrier made:

In [ ]:
# Count how many shipments each carrier made
shipping_df.group_by("Carrier").count().show()

# Count late shipments
shipping_df.filter(shipping_df["Late"] == True).count()

Since this dataset is small, you can easily convert it to pandas and use some of the built-in functions to explore it further.

In [ ]:
pandas_df = shipping_df.to_pandas()
pandas_df.describe()

## ✅ Step 3: Save the data to a table in the database

In [ ]:
shipping_df.write.save_as_table("shipping_logs", mode="overwrite")

## ✅ Challenge Extension (Optional)
Try building a quick prototype in Streamlit that lets a user:
- Search shipping logs by location or keyword
- Display the top locations with shipping issues

In [ ]:
select * FROM shipping_logs;